In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (83.5 kB/s)
Reading package li

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-28 22:21:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.72MB/s    in 0.2s    

2022-11-28 22:21:36 (5.72 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read the first CSV file from an S3 bucket
from pyspark import SparkFiles

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz'

spark.sparkContext.addFile(url)

ar1_df = spark.read.option('encoding', 'UTF-8').csv(SparkFiles.get(''), sep='\t', header=True, inferSchema=True)
ar1_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [6]:
# Count the number of rows
print(ar1_df.count())

6908554


In [7]:
ar1_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# Create first table in schema: review_id_table
# review_id (text), customer_id (int), product_id (text), product_parent (int), review_date (date)

# only date is not correct format
from pyspark.sql.functions import to_date

review_id_df = ar1_df.select([ 'review_id', 'customer_id', 'product_id', 'product_parent',  to_date('review_date', 'yyyy-MM-dd').alias('review_date')  ])

In [9]:
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3ARRMDEGED8RD|   22873041|B00KJWQIIC|     335625766| 2015-08-31|
| RQ28TSA020Y6J|   30088427|B013ALA9LA|     671157305| 2015-08-31|
| RUXJRZCT6953M|   20329786|B00PML2GQ8|     982036237| 2015-08-31|
| R7EO0UO6BPB71|   14215710|B001NS0OZ4|     576587596| 2015-08-31|
|R39NJY2YJ1JFSV|   38264512|B00AQMTND2|     964759214| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [10]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
# Now the products table:
# product_id (text), product_title (text)

products_df = ar1_df.select([ 'product_id', 'product_title'])

In [13]:
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00KJWQIIC|Plemo 14-Inch Lap...|
|B013ALA9LA|TP-Link OnHub AC1...|
|B00PML2GQ8|AmazonBasics USB ...|
|B001NS0OZ4|Transcend P8 15-i...|
|B00AQMTND2|Aleratec SATA Dat...|
+----------+--------------------+
only showing top 5 rows



In [28]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [29]:
# Drop duplicates since the same product will likely have duplicate entries
products_df = products_df.dropDuplicates()

In [14]:
# Now create the customers table:
# customer_id (int), customer_count (int)

customers_df = ar1_df.groupby('customer_id').count().withColumnRenamed('count','customer_count')

In [21]:
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|            11|
|   14991350|             6|
|    9002099|             1|
|   35535911|             3|
|    1117644|             1|
+-----------+--------------+
only showing top 5 rows



In [22]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [24]:
from pyspark.sql.types import IntegerType
customers_df.withColumn('customer_count', customers_df.customer_count.cast('int'))

DataFrame[customer_id: int, customer_count: int]

In [16]:
|# since I cannot load in my password file from collab, there is a function to just manually enter passwords at execution
# Make sure to enter the password set in RDS
from getpass import getpass

password = getpass(prompt='Enter RDS postgres password: ')

Enter postgres password: ··········


In [17]:
# IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = 'mypostgresdb.cvmfkzk4qebt.us-east-1.rds.amazonaws.com' # This is my value; please replace with your own
my_aws_port_number = '5432' # Your value is likely the same, but please double check
my_aws_database_name = 'my_amazon_reviews_db' # This is my value; please replace with your own
my_aws_username = 'postgres' # Your value is likely the same, but please double check
my_aws_password = password # This is my value; please replace with your own


In [18]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to append the existing data. 
mode = 'append' 

Now all of the tables can be populated to the RDS postgres database

In [19]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [26]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [31]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)